In [12]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from fastai.io import *
from fastai.conv_learner import *
from fastai.column_data import *
torch.cuda.set_device(0)
import numpy as np
import pandas as pd
import urllib.request
from pathlib import Path

### Load Data

In [20]:
PATH = Path('/mnt/data/extracts/rnns/')
txt = (PATH/'nietzsche.txt').read_text()

### Create Vocab

In [42]:
chars = set(txt)
padding_char = "\0"
chars.add(padding_char)
chars = sorted(list(chars))
c2i = {c:i for i, c in enumerate(chars)}
i2c = {i:c for i, c in enumerate(chars)}
vocab_size = len(c2i)

### Convert Text

In [5]:
inp = [c2i[c] for c in txt]

In [6]:
x1 = [inp[i] for i in range(0, len(inp) - 3, 3)]
x2 = [inp[i] for i in range(1, len(inp) - 3, 3)]
x3 = [inp[i] for i in range(2, len(inp) - 2, 3)]
x4 = [inp[i] for i in range(3, len(inp) - 1, 3)]

In [7]:
x = np.concatenate([np.vstack(x1), np.vstack(x2), np.vstack(x3)], axis=1)
y = np.vstack(x4)

### Build Model

In [55]:
class char_rnn(nn.Module):
    def __init__(self, vocab_size, n_fac, n_hidden):
        super(char_rnn, self).__init__()
        self.emb = nn.Embedding(vocab_size, n_fac)
        self.input = nn.Linear(n_fac, n_fac)
        self.hidden = nn.Linear(n_fac, n_fac)
        self.output = nn.Linear(n_fac, vocab_size)
        
    def forward(self, c1, c2, c3):
        pdb.set_trace()
        x1 = F.relu(self.input(self.emb(c1)))
        x2 = F.relu(self.input(self.emb(c2)))
        x3 = F.relu(self.input(self.emb(c3)))
        
        h = V(torch.zeros(x1.size()))
        h = F.tanh(self.hidden(h + x1))
        h = F.tanh(self.hidden(h + x2))
        h = F.tanh(self.hidden(h + x3))
        
        return F.log_softmax(self.output(h))

In [56]:
md = ColumnarModelData.from_arrays(PATH, [-1], xs=x, y=y, bs=14)

In [57]:
it = iter(md.trn_dl)
c1, c2, c3, c4 = next(it)

In [58]:
ch = char_rnn(vocab_size, 256, 256)

In [60]:
ch.forward(c1, c2, c3)

> <ipython-input-55-045a31c637ad>(11)forward()
-> x1 = F.relu(self.input(self.emb(c1)))
(Pdb) l
  6  	        self.hidden = nn.Linear(n_fac, n_fac)
  7  	        self.output = nn.Linear(n_fac, vocab_size)
  8  	
  9  	    def forward(self, c1, c2, c3):
 10  	        pdb.set_trace()
 11  ->	        x1 = F.relu(self.input(self.emb(c1)))
 12  	        x2 = F.relu(self.input(self.emb(c2)))
 13  	        x3 = F.relu(self.input(self.emb(c3)))
 14  	
 15  	        h = V(torch.zeros(x1.size()))
 16  	        h = F.tanh(self.hidden(h + x1))
(Pdb) x1
[40, 30, 29, 1, 40, 43, 31, 61, 2, 74, 2, 2, 76, 54, 9, 54, 73, 67, 33, 73, 71, 67, 2, 68, 57, 68, 72, 69, 73, 60, 61, 2, 65, 61, 68, 69, 71, 2, 2, 2, 71, 72, 61, 2, 75, 55, 67, 68, 54, 72, 8, 54, 2, 62, 57, 68, 67, 71, 54, 2, 66, 9, 61, 2, 58, 58, 62, 58, 58, 68, 67, 72, 67, 56, 66, 2, 69, 73, 62, 2, 73, 76, 56, 73, 78, 54, 2, 74, 65, 69, 57, 61, 71, 57, 58, 58, 73, 44, 73, 2, 75, 55, 67, 67, 62, 58, 54, 2, 72, 66, 2, 73, 57, 59, 1, 67, 67, 54, 68,

(Pdb) c1

 58
 74
 65
 68
 61
 68
  2
 56
 68
  2
 58
 69
 67
  2
[torch.cuda.LongTensor of size 14 (GPU 0)]

(Pdb) n
TypeError: torch.index_select received an invalid combination of arguments - got (torch.FloatTensor, int, !torch.cuda.LongTensor!), but expected (torch.FloatTensor source, int dim, torch.LongTensor index)
> <ipython-input-55-045a31c637ad>(11)forward()
-> x1 = F.relu(self.input(self.emb(c1)))
(Pdb) self.emb(c1)
*** TypeError: torch.index_select received an invalid combination of arguments - got (torch.FloatTensor, int, !torch.cuda.LongTensor!), but expected (torch.FloatTensor source, int dim, torch.LongTensor index)
(Pdb) q


BdbQuit: 